<img src="Archivos/miad4.png" width=800x>

## Interactividad usando `panel`

En este taller podrás poner en práctica los conceptos aprendidos acerca del uso del paquete `panel`. Usarás `panel` con el fin de crear dashboards como herramienta de apoyo al momento de resolver problemas específicos en un contexto de Analytics.

Instrucciones y condiciones generales: Esta es una actividad calificada y, por lo tanto, debe ser resuelta individualmente.

### Al desarrollar este taller podrás verificar tu progreso en:

1. Programar la funcionalidad de un dashboard en `panel`.
2. Agregar gráficos y widgets para hacer visualizaciones agradables en `panel`.
3. Personalizar dashobards para generar distintas visualizaciones interactivas en `panel`.

In [108]:
import panel as pn
import plotly.graph_objects as go
import pandas as pd
pn.extension('plotly')

## Pregunta 1

Con la base de datos *COVID-19.csv* crea un dashboard en `panel` que:

1. Permita crear un diagrama de barras horizontal que muestre los casos de COVID-19 de acuerdo a la selección de los siguentes criterios:<br />
    a. Casos o muertes.<br />
    b. Periodo: Histórico, Últimos 7 días y Últimas 24 horas.<br />
    Pista: utilice un control de tipo `RadioButtonGroup` para cada uno de los dos criterios.<br />
2. Contenga un diagrama de barras horizontal de las muertes por millón de personas.
3. Permita visualizar a tráves de un diagrama de barras horizontal los casos en cada una de las regiones de la Organización Munidal de la Salud.

Nota: a excpeción de las regiones, muestre los 10 mayores valores en cada gráfico.

| <center>Variable</center> |<center>Descripción</center> | 
| :-- | :-- | 
| Name | País |
| WHO Region | Región de la OMS |
| Cases - cumulative total | Total de casos hasta el momento |
| Cases - cumulative total per 1 million population | Número de casos acumulado por millón de personas |
| Cases - newly reported in last 7 days | Número de casos nuevos en los últimos 7 días |
| Cases - newly reported in last 24 hours | Número de casos nuevos en las últimas 24 horas |
| Deaths - cumulative total | Total de muertes hasta el momento |
| Deaths - cumulative total per 1 million population | Número de muertes acumulado por millón de personas |
| Deaths - newly reported in last 7 days | Número de muertes nuevas en los últimos 7 días |
| Deaths - newly reported in last 24 hours | Número de muertes nuevas en las últimas 24 horas |

In [109]:
data = pd.read_csv('Archivos/COVID-19.csv')
data = pd.DataFrame(data)
data.head()

,Name,WHO Region,Cases - cumulative total,Cases - cumulative total per 1 million population,Cases - newly reported in last 7 days,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 1 million population,Deaths - newly reported in last 7 days,Deaths - newly reported in last 24 hours,Transmission Classification
0,United States of America,Americas,26547977,80204.73,871365,129961,455735,1376.83,22562,3549,Community transmission
1,India,South-East Asia,10826363,7845.17,80180,12059,154996,112.32,722,78,Clusters of cases
2,Brazil,Americas,9447165,44444.82,328652,50872,230034,1082.21,7368,1239,Community transmission
3,Russian Federation,Europe,3967281,27185.36,116842,16048,76661,525.31,3479,432,Clusters of cases
4,The United Kingdom,Europe,3929839,57888.79,133747,18262,112092,1651.18,6521,828,Community transmission


In [110]:
criteria_selector = pn.widgets.RadioButtonGroup(options=['Casos', 'Muertes'], value='Casos', name='Seleccionar Criterio')
period_selector = pn.widgets.RadioButtonGroup(options=['Histórico', 'Últimos 7 días', 'Últimas 24 horas'], value='Histórico', name='Seleccionar Periodo')
custom_theme = pn.template.DarkTheme()

In [111]:
def update_plot(criterio, periodo):
    # Assuming 'data' is available globally or in the function's scope
    global data

    # Define a dictionary to map criteria and periods to data columns
    column_mapping = {
        'Casos': {
            'Histórico': 'Cases - cumulative total',
            'Últimos 7 días': 'Cases - newly reported in last 7 days',
            'Últimas 24 horas': 'Cases - newly reported in last 24 hours'
        },
        'Muertes': {
            'Histórico': 'Deaths - cumulative total',
            'Últimos 7 días': 'Deaths - newly reported in last 7 days',
            'Últimas 24 horas': 'Deaths - newly reported in last 24 hours'
        }
    }

    # Use the dictionary to get the corresponding data column
    y_data = data[column_mapping[criterio][periodo]]

    # Create the plot
    fig = go.Figure(go.Bar(x=y_data, y=data['Name'], orientation='h'))
    fig.update_layout(title=f'{criterio} de COVID-19 - {periodo}', xaxis_title=f'{criterio}', yaxis_title='País')

    return fig


In [112]:
dashboard = pn.Row(
    pn.Column('COVID-19 Dashboard', criteria_selector, period_selector),
    pn.panel(update_plot(criterio='Casos', periodo='Histórico'))
)

@pn.depends(criteria_selector.param.value, period_selector.param.value)
def update_dashboard(criterio, periodo):
    return update_plot(criterio, periodo)

dashboard[1] = pn.panel(update_dashboard)
dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'ea2039ed-bb28-4d85-9615-df60371152ae': {'version…

In [113]:
criteria_selector = pn.widgets.RadioButtonGroup(options=['Casos', 'Muertes', 'Muertes por millón'], value='Casos', name='Seleccionar Criterio')
period_selector = pn.widgets.RadioButtonGroup(options=['Histórico', 'Últimos 7 días', 'Últimas 24 horas'], value='Histórico', name='Seleccionar Periodo')

In [114]:
def update_plot(criterio, periodo):
    # Assuming 'data' is available globally or in the function's scope
    global data

    # Define a dictionary to map criteria and periods to data columns
    column_mapping = {
        'Casos': {
            'Histórico': 'Cases - cumulative total',
            'Últimos 7 días': 'Cases - newly reported in last 7 days',
            'Últimas 24 horas': 'Cases - newly reported in last 24 hours'
        },
        'Muertes': {
            'Histórico': 'Deaths - cumulative total',
            'Últimos 7 días': 'Deaths - newly reported in last 7 days',
            'Últimas 24 horas': 'Deaths - newly reported in last 24 hours'
        },
        'Muertes por millón': 'Deaths - cumulative total per 1 million population'
    }

    # Use the dictionary to get the corresponding data column
    y_data = data[column_mapping[criterio][periodo]] if isinstance(column_mapping[criterio], dict) else data[column_mapping[criterio]]

    # Create the plot
    fig = go.Figure(go.Bar(x=y_data, y=data['Name'], orientation='h'))
    fig.update_layout(title=f'{criterio} de COVID-19 - {periodo}', xaxis_title=f'{criterio}', yaxis_title='País')

    return fig

In [115]:
dashboard = pn.Row(
    pn.Column('## COVID-19 Dashboard', criteria_selector, period_selector),
    pn.panel(update_plot(criterio='Casos', periodo='Histórico'))
)

@pn.depends(criteria_selector.param.value, period_selector.param.value)
def update_dashboard(criterio, periodo):
    return update_plot(criterio, periodo)

dashboard[1] = pn.panel(update_dashboard)

dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'b271f902-d08f-46eb-876a-37820c8d809a': {'version…

In [116]:
#Implente su respuesta del punto 3
regiones = data['WHO Region'].unique().tolist()

In [117]:
criteria_selector = pn.widgets.RadioButtonGroup(options=['Casos', 'Muertes', 'Muertes por millón'], value='Casos', name='Seleccionar Criterio')
period_selector = pn.widgets.RadioButtonGroup(options=['Histórico', 'Últimos 7 días', 'Últimas 24 horas'], value='Histórico', name='Seleccionar Periodo')
region_selector = pn.widgets.Select(options=regiones, value=regiones[0], name='Seleccionar Región')

In [118]:
def update_plot(criterio, periodo, region):
    # Assuming 'data' is available globally or in the function's scope
    global data

    # Define a dictionary to map criteria and periods to data columns
    column_mapping = {
        'Casos': {
            'Histórico': 'Cases - cumulative total',
            'Últimos 7 días': 'Cases - newly reported in last 7 days',
            'Últimas 24 horas': 'Cases - newly reported in last 24 hours'
        },
        'Muertes': {
            'Histórico': 'Deaths - cumulative total',
            'Últimos 7 días': 'Deaths - newly reported in last 7 days',
            'Últimas 24 horas': 'Deaths - newly reported in last 24 hours'
        },
        'Muertes por millón': 'Deaths - cumulative total per 1 million population'
    }

    # Use the dictionary to get the corresponding data column
    y_data = data[data['WHO Region'] == region][column_mapping[criterio][periodo]] if isinstance(column_mapping[criterio], dict) else data[data['WHO Region'] == region][column_mapping[criterio]]

    # Create the plot
    fig = go.Figure(go.Bar(x=y_data, y=data[data['WHO Region'] == region]['Name'], orientation='h'))
    fig.update_layout(title=f'{criterio} de COVID-19 - {periodo} en {region}', xaxis_title=f'{criterio}', yaxis_title='País')

    return fig

In [119]:
dashboard = pn.Row(
    pn.Column('COVID-19 Dashboard', criteria_selector, period_selector, region_selector),
    pn.panel(update_plot(criterio='Casos', periodo='Histórico', region=regiones[0]))
)

@pn.depends(criteria_selector.param.value, period_selector.param.value, region_selector.param.value)
def update_dashboard(criterio, periodo, region):
    return update_plot(criterio, periodo, region)

dashboard[1] = pn.panel(update_dashboard)

dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'09ae0b3d-5df3-4ef8-b112-da6e6637ec4b': {'version…

## Créditos

**Autor(es)**: Juan David Reyes Jaimes, Diego Alejandro Cely Gomez

**Última actualización:** 26/08/2021